<a href="https://colab.research.google.com/github/saintblue97/DeepNN_Optiver/blob/main/optiver_aeguiarte_tseries_lstmv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    from google.colab import drive; drive.mount('/content/drive')
except Exception as e:
    print("Not in Google Colab")

Not in Google Colab


In [ ]:
%%time
%%capture
%reset -f
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage, xgboost, lightgbm as lgb, librosa

ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
    def __init__(self, lim:'RunTimeLimit'=7200): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
    def ShowTime(self):
        msg = f'Runtime is {time.time()-self.t0:.0f} sec'
        print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 2.23 s, sys: 501 ms, total: 2.73 s
Wall time: 3.09 s


In [ ]:
def is_google_colab():
    """Check if the environment is Google Colab.
    Returns:
        bool: True if in Google Colab, False otherwise.
    """
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
if is_google_colab():
    !pip install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
    !mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
    !cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
    !cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
    !chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
    !kaggle config set -n competition -v optiver-trading-at-the-close        # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
    !kaggle competitions download >> log                          # download competition dataset as a zip file
    !unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
    # !kaggle competitions leaderboard --show                       # print public leaderboard

In [ ]:
if is_google_colab():
    file_path= ''
else:
    file_path = '/kaggle/input/optiver-trading-at-the-close/'

In [ ]:
tmr = Timer()

⏳ started. You have 7200 sec. Good luck!


In [ ]:
file_path

'/kaggle/input/optiver-trading-at-the-close/'

In [ ]:
sample_submission_path = file_path + 'example_test_files/sample_submission.csv'

sample_submission = pd.read_csv(sample_submission_path); sample_submission

,time_id,row_id,target
0,26290,478_0_0,0
1,26290,478_0_1,0
...,...,...,...
32998,26454,480_540_198,0
32999,26454,480_540_199,0


## Imports

In [ ]:
pd.set_option('display.max_rows', 60)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold, train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ARDRegression, BayesianRidge, LinearRegression, Ridge, Lars
from tqdm import tqdm

In [ ]:
from operator import itemgetter

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Flatten, Dense, Dropout, LSTM, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential

In [ ]:
SAMPLE_SIZE = 500 # Number of examples = SAMPLE_SIZE * 55
RANDOM_STATE = 40
diff_order = 1

# Load Data

In [ ]:
train_data_path = file_path + 'train.csv'
df = pd.read_csv(train_data_path);
df = df.dropna(subset=['target']); df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3.18e+06,1,1.0,1.34e+07,NaN,NaN,1.0,60651.50,1.0,8493.03,1.0,-3.03,0,0_0_0
1,1,0,0,1.67e+05,-1,1.0,1.64e+06,NaN,NaN,1.0,3233.04,1.0,20605.09,1.0,-5.52,0,0_0_1
2,2,0,0,3.03e+05,-1,1.0,1.82e+06,NaN,NaN,1.0,37956.00,1.0,18995.00,1.0,-8.39,0,0_0_2
3,3,0,0,1.19e+07,-1,1.0,1.84e+07,NaN,NaN,1.0,2324.90,1.0,479032.40,1.0,-4.01,0,0_0_3
4,4,0,0,4.48e+05,-1,1.0,1.79e+07,NaN,NaN,1.0,16485.54,1.0,434.10,1.0,-7.35,0,0_0_4


In [ ]:
df.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id'],
      dtype='object')

In [ ]:
test_data_path = file_path + 'example_test_files/test.csv'
df_test  = pd.read_csv(test_data_path); df_test.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id,currently_scored
0,0,478,0,3.75e+06,-1,1.0,1.15e+07,NaN,NaN,1.0,22940.00,1.0,9177.60,1.0,26290,478_0_0,False
1,1,478,0,9.86e+05,-1,1.0,3.85e+06,NaN,NaN,1.0,1967.90,1.0,19692.00,1.0,26290,478_0_1,False
2,2,478,0,5.99e+05,1,1.0,4.36e+06,NaN,NaN,1.0,4488.22,1.0,34955.12,1.0,26290,478_0_2,False
3,3,478,0,2.87e+06,-1,1.0,2.71e+07,NaN,NaN,1.0,16082.04,1.0,10314.00,1.0,26290,478_0_3,False
4,4,478,0,7.40e+05,-1,1.0,8.88e+06,NaN,NaN,1.0,19012.35,1.0,7245.60,1.0,26290,478_0_4,False


In [ ]:
print('Shape of training data =', df.shape)
print('Shape of testing data  =', df_test.shape)

Shape of training data = (5237892, 17)
Shape of testing data  = (33000, 17)


In [ ]:
# Remove NaNs here!!
df_pp = df.dropna()

In [ ]:
df_pp.shape

(2343638, 17)

In [ ]:
def get_grouped_df(original_df:pd.DataFrame):
    grouped_df_lst = []
    incomplete_ts = []
    cnt = 0
    for k, subdf in tqdm(df.groupby(['stock_id', 'date_id'])):
        if subdf.shape[0] >= 55:
            grouped_df_lst.append(subdf)
        else:
            incomplete_ts.append({"stock_id": k[0], "date_id": k[1], "length": len(subdf)})
            #display(subdf.head(20))
            # print(subdf.shape)
    return grouped_df_lst, incomplete_ts

In [ ]:
grp_df_lst, incomplete_ts_lst = get_grouped_df(df_pp)

100%|██████████| 95235/95235 [00:06<00:00, 14647.68it/s]


In [ ]:
len(grp_df_lst)

95232

In [ ]:
grp_df_lst[0].head(5)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3.18e+06,1,1.0,1.34e+07,NaN,NaN,1.0,60651.50,1.0,8493.03,1.0,-3.03,0,0_0_0
191,0,0,10,1.30e+06,1,1.0,1.53e+07,NaN,NaN,1.0,13996.50,1.0,23519.16,1.0,0.39,1,0_10_0
382,0,0,20,1.30e+06,1,1.0,1.53e+07,NaN,NaN,1.0,4665.50,1.0,12131.60,1.0,4.22,2,0_20_0
573,0,0,30,1.30e+06,1,1.0,1.53e+07,NaN,NaN,1.0,55998.00,1.0,46203.30,1.0,5.45,3,0_30_0
764,0,0,40,1.22e+06,1,1.0,1.53e+07,NaN,NaN,1.0,14655.95,1.0,26610.45,1.0,3.17,4,0_40_0


In [ ]:
dataset_folds = KFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
for fold_num, (train_index, test_index) in enumerate(dataset_folds.split(grp_df_lst)):
    XY_train_df_lst = list(itemgetter(*list(train_index))(grp_df_lst))[:SAMPLE_SIZE]
    XY_test_df_lst = list(itemgetter(*list(test_index))(grp_df_lst))[:SAMPLE_SIZE//10]
    break # Get one fold


In [ ]:
len(XY_train_df_lst)

500

In [ ]:
XY_train_df_lst[0].head(5)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3.18e+06,1,1.0,1.34e+07,NaN,NaN,1.0,60651.50,1.0,8493.03,1.0,-3.03,0,0_0_0
191,0,0,10,1.30e+06,1,1.0,1.53e+07,NaN,NaN,1.0,13996.50,1.0,23519.16,1.0,0.39,1,0_10_0
382,0,0,20,1.30e+06,1,1.0,1.53e+07,NaN,NaN,1.0,4665.50,1.0,12131.60,1.0,4.22,2,0_20_0
573,0,0,30,1.30e+06,1,1.0,1.53e+07,NaN,NaN,1.0,55998.00,1.0,46203.30,1.0,5.45,3,0_30_0
764,0,0,40,1.22e+06,1,1.0,1.53e+07,NaN,NaN,1.0,14655.95,1.0,26610.45,1.0,3.17,4,0_40_0


In [ ]:
# Release Memory
if 'grp_df_lst' in vars(): del grp_df_lst
if 'incomplete_ts_lst' in vars(): del incomplete_ts_lst
if 'dataset_folds' in vars(): del dataset_folds

In [ ]:
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        X_orig = X
        self.n_features_in_ = X.shape[1]
        if hasattr(X_orig, "columns"):
            self.feature_names_in_ = np.array(X_orig.columns, dtype=object)
        return self

    def transform(self, X):
        # Only drop columns that exist in X
        columns_to_drop = [col for col in self.columns_to_drop if col in X.columns]
        return X.drop(columns_to_drop, axis=1)

    def get_feature_names_out(self, names=None):
        return ['hola']

    def get_feature_names_out(self, input_features=None):
        if input_features is None:
            return getattr(self, "feature_names_in_",
                           [f"x{i}" for i in range(self.n_features_in_)])
        else:
            if len(input_features) != self.n_features_in_:
                raise ValueError("Invalid number of features")
            if hasattr(self, "feature_names_in_") and not np.all(
                self.feature_names_in_ == input_features
            ):
                raise ValueError("input_features ≠ feature_names_in_")
            return input_features

class AddRatios(BaseEstimator, TransformerMixin):
    def __init__(self, col_pair_lst:list):
        self.col_pair_lst = col_pair_lst

    def fit(self, X, y=None):
        X_orig = X
        self.n_features_in_ = X.shape[1]
        if hasattr(X_orig, "columns"):
            self.feature_names_in_ = np.array(X_orig.columns, dtype=object)
        return self

    def transform(self, X):
        X_ = X.copy()
        for col1, col2 in self.col_pair_lst:
            non_zero_mask = X_[col2] != 0
            X_[f'{col1}_to_{col2}_ratio'] = np.where(non_zero_mask, X_[col1] / X_[col2], 0)
        return X_

    def get_feature_names_out(self, input_features=None):
        if input_features is None:
            return getattr(self, "feature_names_in_",
                           [f"x{i}" for i in range(self.n_features_in_)])
        else:
            if len(input_features) != self.n_features_in_:
                raise ValueError(f"Invalid number of features.\n input_features: {input_features}\n self.n_features_in_:{self.n_features_in_}")
            if hasattr(self, "feature_names_in_") and not np.all(
                self.feature_names_in_ == input_features
            ):
                raise ValueError("input_features ≠ feature_names_in_")
            return input_features + [f'{col1}_to_{col2}_ratio' for col1, col2 in self.col_pair_lst]

In [ ]:
# Set up preprocessing pipeline
ratios_cols = [
      ('reference_price', 'wap'),
      ('bid_price', 'wap'),
      ('ask_price', 'wap'),
      ]
drop_cols = [
    'near_price','far_price','time_id',
    'bid_price', 'ask_price', 'wap',
    'row_id',
    ]

ds_transformer = ColumnTransformer([
        ('select_cols', 'drop', drop_cols),
        ('imputer', SimpleImputer(strategy='constant', fill_value=0), make_column_selector(dtype_include=np.number)),
        ],
    )
pp_pipeline = Pipeline([
    ('add_ratios', AddRatios(ratios_cols)),
    #('ds_transformer', ds_transformer),
    ('drop_cols', DropColumns(drop_cols)),
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    # ('imputer', SimpleImputer(strategy='median')),
    # ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
])

In [ ]:
len(XY_train_df_lst)

500

In [ ]:
def preprocess_dataset(target_df:pd.DataFrame, diff_order:int=1):
    drop_cols = [
    'near_price', 'far_price', 'time_id',
    'seconds_in_bucket', 'stock_id', 'date_id',
    'row_id',
    ]
    valid_cols = [
        #'stock_id', 'date_id', 'seconds_in_bucket', 'row_id', # Not needed after grouping per-stock time series
        'imbalance_size', 'imbalance_buy_sell_flag',
        'reference_price', 'far_price', 'near_price',
        'matched_size', 'bid_price', 'bid_size', 'ask_price', 'ask_size',
        'wap',
        ]
    pp_target_df = target_df.copy()
    pp_target_df.fillna(0, inplace=True)
    if 'target' in target_df:
        valid_cols.append('target')
    pp_target_df = pp_target_df[valid_cols]
    ####

    # Differencing Time series Dataset
    for _ in range(diff_order):
        pp_target_df = pp_target_df.diff()
    pp_target_df.dropna(inplace=True)
    return pp_target_df

In [ ]:
def get_lag_features(target_df:pd.DataFrame, wsize, wstride, num_ftime_steps=1):
    # Each sample is a sequence of historical time steps
    (N, p) = target_df.shape
    init_index = wsize * wstride - num_ftime_steps
    last_index = N - num_ftime_steps - wsize
    lagtX_lst, lagtY_lst = [], []
    for i in range(init_index, last_index):
        lagtX_lst.append(target_df.iloc[(i-wsize+1):i, :].values)
        lagtY_lst.append(target_df.iloc[i:i+wsize-1]['target'].values)
    lagtX, lagtY = np.array(lagtX_lst), np.array(lagtY_lst)
    return lagtX, lagtY

In [ ]:
def get_ts_dataset(dataset_df_lst:list, num_htime_steps:int=55,  diff_order:int=1, shuffle:bool=True, batch_size=256):
    XY_dataset = None
    num_features = None
    for XY_df in tqdm(XY_train_df_lst):
        # Preprocess XY DataFrame
        XY_df_pp = preprocess_dataset(XY_df, diff_order)
        if num_features is None:
            num_features = XY_df_pp.shape[1]
        # Create Lag Features
        lagX, lagY = get_lag_features(XY_df_pp, wsize=5, wstride=1)
        # Create TF Dataset
        XYtrain_ds = tf.data.Dataset.from_tensor_slices((lagX.copy().astype('float32'), lagY.astype('float32')))
        if XY_dataset is None:
            XY_dataset = XYtrain_ds
        else:
            XY_dataset = XY_dataset.concatenate(XYtrain_ds)#.batch(256)#.prefetch(tf.data.AUTOTUNE)
    if shuffle:
        return XY_dataset.shuffle(4, seed=0, reshuffle_each_iteration=True).batch(batch_size).prefetch(tf.data.AUTOTUNE), num_features
    else:
        return XY_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE), num_features
        #return XY_dataset, num_features

In [ ]:
XY_dataset, num_features = get_ts_dataset(XY_train_df_lst,
                                          diff_order=1,
                                          shuffle=True
                                         )

100%|██████████| 500/500 [00:05<00:00, 84.19it/s]


In [ ]:
num_features

12

In [ ]:
for x_array, y_array in XY_dataset.as_numpy_iterator():
    print(x_array.shape)
    print(y_array.shape)
    break

(256, 4, 12)
(256, 4)


In [ ]:
batch_size = 256

# Make submission

In [ ]:
def buid_model(num_features):
    tf.random.set_seed(0)
    Init = keras.initializers.GlorotUniform(seed=0)
    m = Sequential([
        LSTM(150, return_sequences=True,
             input_shape=[None, num_features],
             name='LSTM1', kernel_initializer=Init, recurrent_initializer=Init),
        Dropout(.2, name='d1'),
        LSTM(100, name='LSTM2', kernel_initializer=Init, recurrent_initializer=Init),
        Dropout(.2, name='d2'),
        Dense(1, name='out', kernel_initializer=Init)
        ], name='Optiver_Model') # we build Ny forecasts

    m.compile(optimizer='adam',
              #optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, use_ema=True),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()],#PearsonCorrMetric()],
              )
    m.summary()
    return m

In [ ]:
model = buid_model(num_features=num_features)

Model: "Optiver_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 LSTM1 (LSTM)                (None, None, 150)         97800     
                                                                 
 d1 (Dropout)                (None, None, 150)         0         
                                                                 
 LSTM2 (LSTM)                (None, 100)               100400    
                                                                 
 d2 (Dropout)                (None, 100)               0         
                                                                 
 out (Dense)                 (None, 1)                 101       
                                                                 
Total params: 198301 (774.61 KB)
Trainable params: 198301 (774.61 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
%%time
def lr_scheduler(epoch, lr):
    decay_rate = 0.1
    decay_step = 3
    if epoch % decay_step == 0 and epoch:
        return lr * np.exp(-(decay_rate * np.floor(epoch / decay_step)))
    return lr

callbacks = [
    keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)
]

# Train model
hist = model.fit(XY_dataset,
                 batch_size=256,
                 epochs=40,
                 callbacks=callbacks,
                 )


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/40
86/86 [==============================] - 8s 41ms/step - loss: 18.7019 - mean_absolute_error: 3.0119 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/40
86/86 [==============================] - 4s 41ms/step - loss: 18.6601 - mean_absolute_error: 3.0072 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/40
86/86 [==============================] - 4s 41ms/step - loss: 18.6488 - mean_absolute_error: 3.0068 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0009048374610134307.
Epoch 4/40
86/86 [==============================] - 4s 41ms/step - loss: 18.6403 - mean_absolute_error: 3.0057 - lr: 9.0484e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0009048374486155808.
Epoch 5/40
86/86 [==============================] - 4s 41ms/step - loss: 18.6359 - mean_absolute_err

In [ ]:
def get_initial_future_vals(target_df:pd.DataFrame, diff_order:int):
    # Get initial future values according to diff order
    init_values_all = target_df.iloc[-diff_order:]['target'].values
    if diff_order > 1:
        init_future_vals = []
        first_elem = init_values_all[0]
        for i in range(diff_order-1):
            init_future_vals.append(first_elem)
            init_vals_tmp = np.diff(init_values_all[i:])
            first_elem = init_vals_tmp[0]
            #print(f"i={i} - init_values_all:{init_values_all} init_future_vals: {init_future_vals} init_vals_tmp: {init_vals_tmp}")
        return np.concatenate([np.array(init_future_vals), init_vals_tmp])
    else:
        return init_values_all[0]

In [ ]:
def pp_predictions(init_future_values:np.ndarray, predictions, diff_order:int):
    py_preds = np.concatenate([init_future_values.ravel(), predictions])
    py_preds = np.cumsum(py_preds, axis=0)
    return py_preds[diff_order:]

In [ ]:
# batch_size = 4
# XYtest_dataset = None
# for XY_test_df in tqdm(XY_test_df_lst):
#     # Preprocess XY DataFrame
#     Xtest_array = preprocess_dataset(XY_test_df, diff_order)
#     #print(f"Xtest_array.shape: {Xtest_array.shape}")
#     # Create TF Dataset
#     XYtest_ds = tf.data.Dataset.from_tensor_slices((Xtest_array.values, Xtest_array.values[:,-1]))
#     if XYtest_ds is None:
#         raise ValueError
#     if XYtest_dataset is None:
#         XYtest_dataset = copy(XYtest_ds)
#     else:
#         XYtest_dataset = XYtest_dataset.concatenate(XYtest_ds)
#     #break
# XYtest_dataset = XYtest_dataset.shuffle(4, seed=0, reshuffle_each_iteration=True).batch(256).prefetch(tf.data.AUTOTUNE)

In [ ]:
def get_narrow_window_preds(model, target_df_lst, diff_order:int):
    y_preds = None
    y_true_all = None
    for target_df in tqdm(target_df_lst):
        y_true_all = np.concatenate([y_true_all, target_df['target'].values]) if y_true_all is not None else target_df['target'].values
        print(f"y_true_all.shape:{y_true_all.shape}")
        # Get initial future values
        xtest_init_vals = get_initial_future_vals(target_df, diff_order)
        # Preprocess target DataFrame
        target_df_pp = preprocess_dataset(target_df, diff_order)
        # Build TF Dataset
        target_ds = tf.data.Dataset.from_tensor_slices(target_df_pp.values.ravel())
        target_ds= target_ds.window(12, shift=12)
        target_ds_batched = target_ds.flat_map(lambda window_ds: window_ds.batch(12))

        #target_ds_batched = target_ds.flat_map(lambda x:x.batch(1))
        batch_preds = None
        for batch_ds in target_ds_batched:
            #print(tf.expand_dims(batch, axis=0).numpy().shape)
            pred = model.predict(tf.expand_dims(batch_ds[np.newaxis, ...], axis=0), verbose=0)
            batch_preds = np.concatenate([batch_preds, pred]) if batch_preds is not None else pred
        batch_preds_pp = pp_predictions(xtest_init_vals, batch_preds.ravel(), diff_order)
        y_preds = np.concatenate([y_preds, batch_preds_pp]) if y_preds is not None else batch_preds_pp
        print(f"len(y_preds):{len(y_preds)}")
        break
    return y_true_all, np.array(y_preds)

In [ ]:
y_true, all_preds = get_narrow_window_preds(model, XY_test_df_lst, diff_order)

  0%|          | 0/50 [00:00<?, ?it/s]

y_true_all.shape:(55,)


  0%|          | 0/50 [00:03<?, ?it/s]

len(y_preds):54


In [ ]:
mean_absolute_error(y_true, all_preds)

ValueError: Found input variables with inconsistent numbers of samples: [2750, 2700]

In [ ]:
all_preds.shape

(50, 1)

In [ ]:
y_true.shape

(2750,)

In [ ]:
XY_test_df = XY_test_df_lst[0].copy()

In [ ]:
xtest_init_vals = get_initial_future_vals(XY_test_df, diff_order)

In [ ]:
xtest_init_vals

7.0405006

In [ ]:
XY_test_df.shape

(55, 17)

In [ ]:
Xtest_df_pp = preprocess_dataset(XY_test_df, diff_order) ; Xtest_df_pp.to_numpy().shape

(54, 12)

In [ ]:
#test_ds = tf.data.Dataset.from_tensor_slices(tf.expand_dims(Xtest_df_pp.to_numpy(), axis=0)).batch(1)
test_ds = tf.data.Dataset.from_tensor_slices(Xtest_df_pp.to_numpy().ravel())#.batch(1)

In [ ]:
test_ds

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.float64, name=None)>

In [ ]:
for elem in list(test_ds.as_numpy_iterator()):
    print(elem.shape)
    #break

(54, 12)


In [ ]:
test_ds= test_ds.window(12, shift=12)
test_ds = test_ds.flat_map(lambda window_ds: window_ds.batch(12))

In [ ]:
cnt = 0
for window_ds in test_ds:
    print(tf.expand_dims(window_ds[np.newaxis, ...], axis=0).shape)
    cnt += 1
print(cnt)

(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
(1, 1, 12)
54


In [ ]:
cnt = 0
for etensor in test_ds:
    print(type(etensor))
    for elem in etensor:
        print(elem.shape)
        cnt += 1
    if cnt > 5:
        break
print(cnt)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
12


In [ ]:
test_ds

<_WindowDataset element_spec=DatasetSpec(TensorSpec(shape=(12,), dtype=tf.float64, name=None), TensorShape([]))>

In [ ]:
cnt = 0
for elem in test_ds.batch(1):
#     print(elem.shape)
    cnt += 1
#     if cnt > 5:
#         break
print(cnt)

5


In [ ]:
tf.data.Dataset.from_tensor_slices(tf.constant(np.array([[1,2,3],[4,5,6]]), shape=(2,3)))

<_TensorSliceDataset element_spec=TensorSpec(shape=(3,), dtype=tf.int64, name=None)>

In [ ]:
len(all_preds)

54

In [ ]:
all_preds[0]

array([6.95])

In [ ]:
len(all_predictions)

50

In [ ]:
all_predictions[0]

7.832714576383209

In [ ]:
len(Ytest_all)

2750

In [ ]:
# if not is_google_colab():
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [ ]:
from sklearn.metrics import mean_absolute_error
import pandas as pd

if is_google_colab():

    # Lists to store true and predicted targets
    true_targets = []
    predicted_targets = []

    for (test, revealed_targets, sample_prediction) in iter_test:
        # Predict target values
        # print(f"test.shape: {test.shape}")
        # print(f"test.columns: {test.columns}")
        # display(test)

        xtest_init_vals = get_initial_future_vals(test, diff_order)
        test_df_pp = preprocess_dataset(test, diff_order)
        preds = model.predict(test_df_pp.values[np.newaxis,...])
        predictions = pp_predictions(xtest_init_vals, preds.ravel(), diff_order)

#         test_dataset = get_tf_dataset(test)
#         predictions = model.predict(test_dataset)
        # print(f"predictions.shape:{predictions.shape}")

        # Only take as many predictions and true targets as the minimum of the two
        min_len = min(len(predictions), len(revealed_targets))

        # Trim down the predictions list so that it is the same size as the revealed targets
        current_predictions = predictions[:min_len]
        current_true_targets = revealed_targets['revealed_target'].values[:min_len]

        # Extend the lists of true targets and predictions with the ones from the current batch
        true_targets.extend(current_true_targets)
        predicted_targets.extend(current_predictions)

        # Make the actual prediction and add to sample_prediction df
        sample_prediction['target'] = predictions

        # Submit the predictions for this chunk of test data
        env.predict(sample_prediction)

    # Convert lists to dataframes
    df_true = pd.DataFrame(true_targets, columns=['true_target'])
    df_pred = pd.DataFrame(predicted_targets, columns=['predicted_target'])

    # Calculate and print the MAE
    mae = mean_absolute_error(df_true.fillna(0), df_pred)
    print("Mean Absolute Error:", mae)
    # Mean Absolute Error: 5.709755347938988 - 10/26/2023

In [ ]:
if not is_google_colab():
    counter = 0
    for (test, revealed_targets, sample_prediction) in iter_test:
        # Predict target values
        xtest_init_vals = get_initial_future_vals(test, diff_order)
        test_df_pp = preprocess_dataset(test, diff_order)
        preds = model.predict(test_df_pp.values[np.newaxis,...])
        predictions = pp_predictions(xtest_init_vals, preds.ravel(), diff_order)

#         test_dataset = get_tf_dataset(test)
#         predictions = model.predict(test_dataset)
        print(f"predictions.shape:{predictions.shape}")
        print(revealed_targets.head())
        sample_prediction['target'] = predictions
        env.predict(sample_prediction)
        counter += 1

In [ ]:
tmr.ShowTime()    # measure Colab's runtime.